Last year we lost millions of dollars on our casino. Today we suggest that you play in an updated one.

- casino_server.py

golden-antelope.quals.2018.volgactf.ru:8888

```
$ nc golden-antelope.quals.2018.volgactf.ru 8888                                                                                 Solve a puzzle: find an x such that 26 last bits of SHA1(x) are set, len(x)==29 and x[:24]=='ad0a95ef655bd426ce7b4d4d'
```

Ah, a proof of work calculation. There may be a way to do this efficiently with hashcat, but we've gone for a C# based brute-forcer. With some help from Google Cloud, this helps solve the POS in a few minutes.

```c#
private static string FindMatchingData(string prefix)
{
    if (prefix.Length == 29)
    {
        var result = calculateHash(prefix);
        // last 26 bits set: 03 ff ff ff
        if (
            ((result[16] & 0x03) == 0x03)) &&
            ((result[17] & 0xff) == 0xff) &&
            ((result[18] & 0xff) == 0xff) &&
            ((result[19] & 0xff) == 0xff))
        {
            Console.WriteLine("Input: " + prefix);
            Console.WriteLine("Output: " + String.Join(" ", result.Select(x => x.ToString("X2"))));
            return prefix;
        }
        return null;
    }

    // limit the search to printable characters
    for (int i = 32; i < 127; i++)
    {
        var r = FindMatchingData(prefix + (char)i);
        if (r != null) return r;
    }
    
    return null;
}
```

Enter the value and claim your fla...

```
Welcome to Guess-a-number game at VolgaCTF "Golden Antelope" Grand Casino!
Your stack: 30

Guess a number in range [0, 256):
0
Wrong. The number was 148. Your stack: 29

```

Oh. 

Instead, take a look at the provided `casino_server.py`, where we find a set of LFSRs:

```python
class Generator:
    def __init__(self, state):
        self.state = state

    def next_state(self, idxs):
        self.idxs = idxs
        y = 0
        for i in self.idxs:
            y ^= self.state[i]
        out = self.state[31]
        for i in range(31, 0, -1):
            self.state[i] = self.state[i - 1]
        self.state[0] = y

RX = Generator([32 random bits])
RA = Generator([32 random bits])
RB = Generator([32 random bits])

while gambling:
    RX.next_state(X)
    if RX.state[29] == 0:
        RA.next_state(A0)
    else:
        RA.next_state(A1)
    if RX.state[26] == 0:
        RB.next_state(B)
    else:
        RB.next_state(B)
        RB.next_state(B)
    
    next_random_number = some_function_of(RX.state, RA.state, RB.state)
```

To solve this puzzle, we need to recover all 32*3 = 96 bytes of random data. We do this by iteratively building a list of hypotheses for the N rightmost bits of a given generator, and pruning hypotheses that are not consistent with the returned random number. For every next random number, three or four bits of randomness are added, while we retrieve 8 more bits of information. 

We start by retrieving a set of number from the casino:

In [2]:
random_values = [
    148, 100, 34, 241, 145,
    44, 186, 162, 3, 112,
    246, 180, 241, 145, 170,
    21, 136, 140, 3, 154,
    40, 172, 134, 185, 218]

And then prepare hypotheses based on the first random number:

In [5]:
# from casino_server.py
L = [0xf1, 0xef, 0x29, 0xbe, 0xb8, 0xf6, 0x4f, 0xaf, 0xb2, 0x92, 0xe3, 0xfc, 0xc6, 0x72, 0x48, 0xc3,
         0xbf, 0xa0, 0x10, 0xd1, 0x23, 0x34, 0x0c, 0x07, 0x7c, 0xf8, 0xae, 0xe8, 0xc9, 0xe1, 0x38, 0x36,
         0x4c, 0x2c, 0x0b, 0x70, 0x7b, 0xe7, 0xd7, 0xc5, 0xac, 0x57, 0xab, 0xd5, 0x4b, 0x77, 0xa5, 0xce,
         0xee, 0xf4, 0x47, 0x25, 0x8a, 0xf3, 0xfd, 0xbb, 0x5c, 0xe0, 0x2a, 0x19, 0x5d, 0xeb, 0xa6, 0x81,
         0x12, 0x61, 0x59, 0xcf, 0xc8, 0xa8, 0xfe, 0x3e, 0x31, 0x1e, 0x46, 0x7e, 0x3d, 0xd0, 0x3c, 0xc7,
         0xdc, 0x33, 0x8f, 0xca, 0x78, 0x6f, 0x0d, 0x62, 0x9d, 0xd9, 0x89, 0x73, 0x8c, 0x4e, 0xb7, 0xc0,
         0x03, 0x56, 0xb9, 0x79, 0x75, 0xda, 0x6e, 0x1c, 0xff, 0x67, 0x2f, 0xbc, 0x69, 0x91, 0x2b, 0x9b,
         0x7f, 0x17, 0x01, 0xde, 0xfa, 0x4a, 0x02, 0x0e, 0x8b, 0xa9, 0x58, 0x2d, 0xd8, 0xf9, 0x3b, 0xb3, 
         0x49, 0x65, 0xcc, 0xa3, 0xbd, 0x16, 0x21, 0xd3, 0xe5, 0xd6, 0x42, 0x60, 0x4d, 0x20, 0x97, 0x5e, 
         0x2e, 0xe9, 0x18, 0xc2, 0x63, 0x64, 0xf5, 0x6a, 0xd2, 0x68, 0x1b, 0x1f, 0xc4, 0xea, 0x74, 0xa2, 
         0x45, 0x82, 0xb6, 0x32, 0x84, 0xed, 0x50, 0x26, 0xcb, 0x5f, 0x37, 0xa1, 0x15, 0xa4, 0x51, 0x53, 
         0xb4, 0x09, 0xaa, 0x1a, 0x14, 0x43, 0xba, 0xdf, 0x87, 0x66, 0x85, 0x52, 0x3a, 0x28, 0x9a, 0xb1, 
         0x44, 0x9f, 0x96, 0x41, 0xdd, 0x86, 0x88, 0x9e, 0x71, 0xb0, 0x13, 0x98, 0xe4, 0x05, 0xf7, 0x6c, 
         0xb5, 0x93, 0x8e, 0x55, 0xec, 0x8d, 0xf2, 0x6d, 0x9c, 0xa7, 0xad, 0x00, 0x08, 0xf0, 0xe6, 0x6b, 
         0x7a, 0xcd, 0xfb, 0x80, 0x0a, 0x83, 0x27, 0x39, 0x30, 0x06, 0x76, 0x90, 0x94, 0x35, 0x54, 0x04, 
         0x0f, 0xc1, 0x5b, 0x99, 0x11, 0x40, 0x5a, 0xd4, 0xe2, 0x95, 0x3f, 0x22, 0x7d, 0x24, 0x1d, 0xdb]

def H(state):
    return int("".join(map(str, bin(state)[2:].zfill(32)[-1:-9:-1])), 2)

In [18]:
first_value = random_values[0]

# build hypotheses where (H(RX.state) + L[H(RA.state)] + L[H(RB.state)]) % 256 == first entry

hypotheses = [
    # x-seed, x-shift, a-seed, a-shift, b-seed, b-shift
    (x, 0, a, 0, b, 0)
    for x in range(256)
    for a in range(256)
    for b in range(256)
    if ((H(x) + L[H(a)] + L[H(b)]) % 256) == first_value
]

len(hypotheses)

65536

We can now iteratively add hypotheses for the additional bits, and then prune them in the same manner.

In [7]:
def next_hypotheses(hypotheses, next_value):
    new_hypotheses = []
    
    for x, xshift, a, ashift, b, bshift in hypotheses:
        # x_0 is the unshifted (seed) hypothesis, while x_1 is the value of x
        # used for the generation of the current random number. The same applies
        # for a_0/a_1 and b_0/b_1.
        for x_0 in [x, x + 2**(xshift+8)]:
            x_1 = x_0 >> (xshift + 1)
            
            for a_0 in [a, a + 2**(ashift+8)]:
                a_1 = a_0 >> (ashift + 1)
                
                if bin(x_1)[2:].zfill(6)[-6] == "0":  # if RX.state[26] == 0:
                    shift = 1
                    b_opts = [b, b + 2**(bshift+8)]
                else:
                    # if we shift twice, we add two bits, so there are four options
                    shift = 2
                    b_opts = [b, b + 2**(bshift+8), b + 2**(bshift+9), b + 2**(bshift+8) + 2**(bshift+9)]

                for b_0 in b_opts:
                    b_1 = b_0 >> (bshift + shift)
                    true_number = (H(x_1) + L[H(a_1)] + L[H(b_1)]) % 256
                    if true_number == next_value:
                        new_hypotheses.append((x_0, xshift + 1, a_0, ashift + 1, b_0, bshift + shift))

    return new_hypotheses

In [19]:
for rv in random_values[1:]:
    hypotheses = next_hypotheses(hypotheses, rv)
    print("Value: {:3d}, hypotheses left: {:4d}".format(rv, len(hypotheses)))

Value: 100, hypotheses left: 3033
Value:  34, hypotheses left:  150
Value: 241, hypotheses left:   12
Value: 145, hypotheses left:    1
Value:  44, hypotheses left:    1
Value: 186, hypotheses left:    1
Value: 162, hypotheses left:    1
Value:   3, hypotheses left:    1
Value: 112, hypotheses left:    1
Value: 246, hypotheses left:    1
Value: 180, hypotheses left:    1
Value: 241, hypotheses left:    1
Value: 145, hypotheses left:    1
Value: 170, hypotheses left:    1
Value:  21, hypotheses left:    1
Value: 136, hypotheses left:    1
Value: 140, hypotheses left:    1
Value:   3, hypotheses left:    1
Value: 154, hypotheses left:    1
Value:  40, hypotheses left:    1
Value: 172, hypotheses left:    1
Value: 134, hypotheses left:    1
Value: 185, hypotheses left:    1
Value: 218, hypotheses left:    1


One hypothesis left -- nice! We now retrieve the seeds for the generators:

In [22]:
seed_x, shift_x, seed_a, shift_a, seed_b, shift_b = hypotheses[0]
print("X seed: {}".format(seed_x))
print("A seed: {}".format(seed_a))
print("B seed: {}".format(seed_b))
print()
print("Shifts (should be >= 24): X {}, A {}, B {}".format(shift_x, shift_a, shift_b))

X seed: 495831187
A seed: 880281027
B seed: 6873053221420

Shifts (should be >= 24): X 24, A 24, B 36


We then fill these into the setup of `casino_server.py`:
    
```python
RX = Generator([int(i) for i in bin(495831187).split('b')[1][-32:].zfill(32)])
RA = Generator([int(i) for i in bin(880281027).split('b')[1][-32:].zfill(32)])
RB = Generator([int(i) for i in bin(6873053221420).split('b')[1][-32:].zfill(32)])
```

and generate enough random numbers to bring the score to 108:
    
```
Congratulations! Your stack: 108
Your flag: VolgaCTF{T@ke_the_M0n3y_4nd_Run}
```